<a href="https://colab.research.google.com/github/trefftzc/cis677/blob/main/Collective_communications_in_mpi4py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Examples of Collective Communications in MPI4PY

A great reference to collective communications in MPI is available in this web site of the Lawrence Livermore National Lab: https://hpc-tutorials.llnl.gov/mpi/collective_communication_routines/

In [1]:
!pip install mpi4py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 12.5 MB/s eta 0:00:00


# Broadcast

In [2]:
%%writefile bcast.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    data = {'value': 123}
else:
    data = None

data = comm.bcast(data, root=0)

print(f"Process {rank} received data: {data}")

Writing bcast.py


In [3]:
!OMPI_ALLOW_RUN_AS_ROOT=1
!mpiexec --allow-run-as-root -n 4 --oversubscribe python bcast.py

Process 0 received data: {'value': 123}
Process 2 received data: {'value': 123}
Process 1 received data: {'value': 123}
Process 3 received data: {'value': 123}


#Scatter

In [4]:
%%writefile scatter.py
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0:
    send_data = np.arange(size * 2, dtype='i').reshape(size, 2)
else:
    send_data = None

recv_data = np.empty(2, dtype='i')

comm.Scatter(send_data, recv_data, root=0)

print(f"Process {rank} received: {recv_data}")

Writing scatter.py


In [5]:
!OMPI_ALLOW_RUN_AS_ROOT=1
!mpiexec --allow-run-as-root -n 4 --oversubscribe python scatter.py

Process 0 received: [0 1]
Process 2 received: [4 5]
Process 1 received: [2 3]
Process 3 received: [6 7]


# Gather

In [6]:
%%writefile gather.py
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

send_data = np.array([rank * 10, rank * 10 + 1], dtype='i')

if rank == 0:
    recv_data = np.empty(size * 2, dtype='i')
else:
    recv_data = None

comm.Gather(send_data, recv_data, root=0)

if rank == 0:
    print(f"Root process received: {recv_data}")

Writing gather.py


In [7]:
!OMPI_ALLOW_RUN_AS_ROOT=1
!mpiexec --allow-run-as-root -n 4 --oversubscribe python gather.py

Root process received: [ 0  1 10 11 20 21 30 31]


# Reduce

In [8]:
%%writefile reduce.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

local_value = rank + 1
total_sum = comm.reduce(local_value, op=MPI.SUM, root=0)

if rank == 0:
    print(f"Total sum: {total_sum}")

Writing reduce.py


In [9]:
!OMPI_ALLOW_RUN_AS_ROOT=1
!mpiexec --allow-run-as-root -n 4 --oversubscribe python reduce.py

Total sum: 10


# Allreduce

In [10]:
%%writefile allreduce.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

local_value = rank + 1
global_sum = comm.allreduce(local_value, op=MPI.SUM)

print(f"Process {rank} knows the global sum: {global_sum}")

Writing allreduce.py


In [11]:
!OMPI_ALLOW_RUN_AS_ROOT=1
!mpiexec --allow-run-as-root -n 4 --oversubscribe python allreduce.py

Process 0 knows the global sum: 10
Process 2 knows the global sum: 10
Process 1 knows the global sum: 10
Process 3 knows the global sum: 10


# alltoall

In [12]:
%%writefile alltoall.py
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

send_data = np.array([rank * 10 + i for i in range(size)], dtype='i')
recv_data = np.empty(size, dtype='i')

comm.Alltoall(send_data, recv_data)

print(f"Process {rank} sent: {send_data}, received: {recv_data}")

Writing alltoall.py


In [13]:
!OMPI_ALLOW_RUN_AS_ROOT=1
!mpiexec --allow-run-as-root -n 4 --oversubscribe python alltoall.py

Process 2 sent: [20 21 22 23], received: [ 2 12 22 32]
Process 0 sent: [0 1 2 3], received: [ 0 10 20 30]
Process 1 sent: [10 11 12 13], received: [ 1 11 21 31]
Process 3 sent: [30 31 32 33], received: [ 3 13 23 33]
